In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from pathlib import Path
import os
from nltk.tokenize import RegexpTokenizer
import nltk
# from sumy.summarizers.lex_rank import LexRankSummarizer 
# #Plain text parsers since we are parsing through text
# from sumy.parsers.plaintext import PlaintextParser
# #for tokenization
# from sumy.nlp.tokenizers import Tokenizer
from nltk.tokenize.toktok import ToktokTokenizer

from gensim.corpora import Dictionary
from gensim.models.ldamulticore import LdaMulticore
from gensim.models.coherencemodel import CoherenceModel
from gensim.parsing.preprocessing import STOPWORDS as SW
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from pprint import pprint


from gensim.parsing.preprocessing import STOPWORDS as SW
from wordcloud import STOPWORDS
stopwords = set(STOPWORDS)

from functions import *
%load_ext autoreload
%autoreload 2

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option("display.max_colwidth", 200)
pd.set_option('display.expand_frame_repr', False)

In [4]:
os.chdir('/Users/ariadnavargas/Box Sync/i42 Projects/External Projects/Project ECHO/Phase 3. Execution and dissemination/ProjectECHO/')
textfile = Path("data/raw/Critical Care for the COVID-19 Patient/")
graphs = Path("graphs/")
output = Path("outputs/")
models = Path("models/")
clean = Path("data/clean")
video1 = "2020-07-02 08.01.05 Critical Care for the COVID-19 Patient session 123702861_speaker1.txt"
video2 = "2020-07-02 08.01.05 Critical Care for the COVID-19 Patient session 123702861_speaker2.txt"
videoq = "2020-07-02 08.01.05 Critical Care for the COVID-19 Patient session 123702861_q&a.txt"
graph1 = video1.split('.txt')[0]
graph2 = video2.split('.txt')[0]
graphq = videoq.split('.txt')[0]
topic1_s1= video1.split('.txt')[0]+'_speaker_1.html'
topic1_s2= video1.split('.txt')[0]+'_speaker_2.html'
model1_s1 = video1.split('.txt')[0]+'_speaker1.pkl'
model1_s2 = video1.split('.txt')[0]+'_speaker2.pkl'
data1= video1.split('.txt')[0]+'.csv'
data1_speaker1= video1.split('.txt')[0]+'.txt'
data1_speaker1_section1= video1.split('.txt')[0]+'_section1.txt'
data1_speaker1_section2= video1.split('.txt')[0]+'_section2.txt'
data1_speaker1_section3= video1.split('.txt')[0]+'_section3.txt'
data1_speaker2_section1= video2.split('.txt')[0]+'_section1.txt'
data1_speaker2_section2= video2.split('.txt')[0]+'_section2.txt'
data1_speaker2_section3= video2.split('.txt')[0]+'_section3.txt'

data1_speaker1_section1_key= video1.split('.txt')[0]+'_section1_key.txt'
data1_speaker1_section2_key= video1.split('.txt')[0]+'_section2_key.txt'
data1_speaker1_section3_key= video1.split('.txt')[0]+'_section3_key.txt'
data1_speaker2_section1_key= video2.split('.txt')[0]+'_section1_key.txt'
data1_speaker2_section2_key= video2.split('.txt')[0]+'_section2_key.txt'
data1_speaker2_section3_key= video2.split('.txt')[0]+'_section3_key.txt'

dataq_questions= videoq.split('.txt')[0]+'_questions.txt'
dataq_questions_key= videoq.split('.txt')[0]+'_questions_key.txt'

# Speaker 1

## Import text file

In [103]:
# Read text file into a dataframe
df = pd.read_csv(textfile/video1, sep="\t",
                 header=0,
                 names=["text", "minute"],
                 index_col=False)

In [104]:
df

,text,minute
0,All right.,1.160167
1,"Thanks, Doctor Thornton.",1.166833
2,Good morning and good morning before a holiday weekend.,1.183000
3,"So it's good to see some, uh, fases, even though maybe no one's going anywhere for the holiday.",1.232833
4,"I know I'm not, but that's okay.",1.312833
5,"So thanks for joining us this morning, and I I asked a colleague of mine to give a presentation today on, Ah, diabetes.",1.345167
6,"And the reason, um, you know, the Wuhan data suggested that the patients that had the worst outcome were those with cardiovascular disease and, um, you know, age over, you know, 50 60 70 and went ...",1.487333
7,"Um, but what we noticed in our cohort of patients, the majority of patients that did poorly were actually diabetics and in potential obese diabetic.",1.758000
8,"So my question was, you know, what is there?",1.923333
9,Is there something specific about diabetes that's putting these patients at risk and do?,1.985833


In [105]:
# Drop empty rows
df.dropna(
    axis=0,
    how='any',
    thresh=None,
    subset=None,
    inplace=True
)

In [106]:
df.head()

,text,minute
0,All right.,1.160167
1,"Thanks, Doctor Thornton.",1.166833
2,Good morning and good morning before a holiday weekend.,1.183000
3,"So it's good to see some, uh, fases, even though maybe no one's going anywhere for the holiday.",1.232833
4,"I know I'm not, but that's okay.",1.312833


In [107]:
# displaying the number of rows and columns 
rows = df.shape[0] 
cols = df.shape[1] 
print("Rows: " + str(rows)) 
print("Columns: " + str(cols)) 

Rows: 92
Columns: 2


In [108]:
# convert dataframe to list
temp = df.values.tolist()

# for i in temp:
#     print(i[1])

In [109]:
# text=[]
# # remove inner brackets and quotes
# for t in temp:
#     nobrackets = str(t)[1:-1]
#     stripped = nobrackets.strip('"')
#     text.append(str(stripped)[1:-1])

In [110]:
text=[]
time=[]
for i in temp:
    text.append(i[0])
    time.append(i[1])    

In [111]:
# sanity check
for i in range(10):
    print(text[i] + ' Starting time: ' + str(time[i]))

All right. Starting time: 1.1601666666666666
Thanks, Doctor Thornton. Starting time: 1.1668333333333334
Good morning and good morning before a holiday weekend. Starting time: 1.183
So it's good to see some, uh, fases, even though maybe no one's going anywhere for the holiday. Starting time: 1.2328333333333332
I know I'm not, but that's okay. Starting time: 1.3128333333333333
So thanks for joining us this morning, and I I asked a colleague of mine to give a presentation today on, Ah, diabetes. Starting time: 1.3451666666666666
And the reason, um, you know, the Wuhan data suggested that the patients that had the worst outcome were those with cardiovascular disease and, um, you know, age over, you know, 50 60 70 and went up for decade. Starting time: 1.4873333333333332
Um, but what we noticed in our cohort of patients, the majority of patients that did poorly were actually diabetics and in potential obese diabetic. Starting time: 1.758
So my question was, you know, what is there? Starting

In [112]:
section=[]
for i in range(len(text)):
    section.append(0)
len(section)
x=0
for i in range(len(text)):
    if 'family history is positive' not in text[i].lower():
        section[i]=1
        x+=1
    elif 'family history is positive' in text[i].lower():
        section[i]=1
        section[i+1]=2
        x+=1
        print(x)
        break

for i in range(x,len(text)):
    if 'bilateral patch opacities' not in text[i].lower():
        section[i]=2
        x+=1
    elif 'bilateral patch opacities' in text[i].lower():
        section[i]=2
        section[i+1]=3
        x+=1
#         print(text[i])
        break

for i in range(x,len(text)):
#     print(text[i])
    section[i]=3

29


In [113]:
# section

In [114]:
lists = list(zip(section, text, time)) 
df = pd.DataFrame(lists, columns = ['section', 'text', 'time']) 

In [118]:
df

,section,text,time
0,1,All right.,1.160167
1,1,"Thanks, Doctor Thornton.",1.166833
2,1,Good morning and good morning before a holiday weekend.,1.183000
3,1,"So it's good to see some, uh, fases, even though maybe no one's going anywhere for the holiday.",1.232833
4,1,"I know I'm not, but that's okay.",1.312833
5,1,"So thanks for joining us this morning, and I I asked a colleague of mine to give a presentation today on, Ah, diabetes.",1.345167
6,1,"And the reason, um, you know, the Wuhan data suggested that the patients that had the worst outcome were those with cardiovascular disease and, um, you know, age over, you know, 50 60 70 and went ...",1.487333
7,1,"Um, but what we noticed in our cohort of patients, the majority of patients that did poorly were actually diabetics and in potential obese diabetic.",1.758000
8,1,"So my question was, you know, what is there?",1.923333
9,1,Is there something specific about diabetes that's putting these patients at risk and do?,1.985833


In [119]:
start_time = df.groupby(['section'], sort=False)['time'].max()
start_time.head()
# start_time.to_csv(clean/'speaker1_sections.txt', sep='\t', index=False, header=False)

section
1    3.583333
2    4.544333
3    8.680833
Name: time, dtype: float64

In [120]:
copy = df[['text']].copy()
copy
s1 = df['section']==1
df_1 = df.loc[s1]
df_1 = df_1[['text']]
df_1
df_1.to_csv('data/clean/2020-07-02 08.01.05 Critical Care for the COVID-19 Patient session 123702861_speaker1_1.txt', sep='\t', index=False, header=False)

s2 = df['section']==2
df_2 = df.loc[s2]
df_2 = df_2[['text']]
df_2.to_csv(clean/'2020-07-02 08.01.05 Critical Care for the COVID-19 Patient session 123702861_speaker1_2.txt', sep='\t', index=False, header=False)

s3 = df['section']==3
df_3 = df_3.loc[s3]
df_3 = df_3[['text']]
df_3.to_csv(clean/'2020-07-02 08.01.05 Critical Care for the COVID-19 Patient session 123702861_speaker1_3.txt', sep='\t', index=False, header=False)

TypeError: unsupported operand type(s) for /: 'list' and 'str'

# Q&A

## Extract questions from chat

In [37]:
# Read text file into a dataframe

text_list = open('data/raw/Critical Care for the COVID-19 Patient/2020-07-02 08.01.05 Critical Care for the COVID-19 Patient session 123702861_chat.txt').readlines()


In [43]:
for i in range(5):
    print(text_list[i])

08:02:05	 From  Digital Librarian - Project ECHO (Kent) : 

Registration links for all upcoming ECHO Institute New Mexico COVID-19 Sessions:

https://echo.unm.edu/covid-19/institute-nm

08:10:01	 From  Carlos Nolasco : do you think her risk was more increased due to obesity rather than diabetes?

08:11:34	 From  Digital Librarian - Project ECHO (Kent) : 



In [44]:
questions=[]
import re 
  
# initializing list  
test_list = ['? ', '?\n'] 
  
for i in text_list:
    res = [ele for ele in test_list if(ele in i)]
    if bool(res)== True:
#             print(i)
            questions.append(i)

In [46]:
len(questions)
questions

['08:10:01\t From  Carlos Nolasco : do you think her risk was more increased due to obesity rather than diabetes?\n',
 '08:20:28\t From  Carlos Nolasco : any decreased lung expression of ACE2???\n',
 '08:21:52\t From  Mary Zeitner : Do ACE inhibitors lower ACE1 or ACE2, or both?  Thank you.\n',
 '08:31:26\t From  Carlos Nolasco : Taking health and economy disparities out of the discussion, do you have any data suggesting that Hispanics may be at a higher risk of hospitalization or worse outcomes due to their known increased risk for metabolic syndrome and risk for diabetes?\n',
 '08:39:03\t From  Catherine Torres : Do these patients need to see endocrinology and cardiology outpatient one they are discharged?\n',
 '08:39:55\t From  Shanpin Fanchiang : Will the association between ACE2i and the outcome of COVID-19 supports giving non-diabetes patients with COVID-19 ACE2i?  \n',
 'Why ECHO to Address COVID-19 & Diabetes in Primary Care?\n']

In [56]:
questions_clean=[]
for q in questions:
    try:
        clean=q.split(':')
        print(clean)
        questions_clean.append(clean[3])
    except Exception as e:
        print(e)

['08', '10', '01\t From  Carlos Nolasco ', ' do you think her risk was more increased due to obesity rather than diabetes?\n']
['08', '20', '28\t From  Carlos Nolasco ', ' any decreased lung expression of ACE2???\n']
['08', '21', '52\t From  Mary Zeitner ', ' Do ACE inhibitors lower ACE1 or ACE2, or both?  Thank you.\n']
['08', '31', '26\t From  Carlos Nolasco ', ' Taking health and economy disparities out of the discussion, do you have any data suggesting that Hispanics may be at a higher risk of hospitalization or worse outcomes due to their known increased risk for metabolic syndrome and risk for diabetes?\n']
['08', '39', '03\t From  Catherine Torres ', ' Do these patients need to see endocrinology and cardiology outpatient one they are discharged?\n']
['08', '39', '55\t From  Shanpin Fanchiang ', ' Will the association between ACE2i and the outcome of COVID-19 supports giving non-diabetes patients with COVID-19 ACE2i?  \n']
['Why ECHO to Address COVID-19 & Diabetes in Primary Care

In [57]:
# for q in questions_clean:
#     print(q)
len(questions_clean)

6

## Extract answers from transcript

In [61]:
# Read text file into a dataframe
df = pd.read_csv(textfile/videoq, sep="\t",
                 header=0,
                 names=["text", "minute"],
                 index_col=False)

In [62]:
df

,text,minute
0,That's a great question.,28.607333
1,"Um, and I'll let Matt add here too, because I was wondering because I've seen a few that have just had, um, obesity.",28.629167
2,"We have, Ah, a 33 year old, uh, obese gentlemen, 56 BMI.",28.744833
3,On ECMO,28.831667
4,"Who didn't have diabetes, And we haven't had a 23 year old admitted that didn't have diabetes.",28.859000
5,He was just a little portly.,28.963167
6,"Not much, but he you know, 23.",28.985833
7,"So I think the diabetes, um, is a risk factor for some, but I think obesity really adds into that.",29.026167
8,"I don't know Matt if you have any other, uh, comments on that.",29.136000
9,"Yeah, I agree with you, Michelle.",29.214000


In [63]:
# Drop empty rows
df.dropna(
    axis=0,
    how='any',
    thresh=None,
    subset=None,
    inplace=True
)

In [64]:
df.head()

,text,minute
0,That's a great question.,28.607333
1,"Um, and I'll let Matt add here too, because I was wondering because I've seen a few that have just had, um, obesity.",28.629167
2,"We have, Ah, a 33 year old, uh, obese gentlemen, 56 BMI.",28.744833
3,On ECMO,28.831667
4,"Who didn't have diabetes, And we haven't had a 23 year old admitted that didn't have diabetes.",28.859000


In [65]:
# displaying the number of rows and columns 
rows = df.shape[0] 
cols = df.shape[1] 
print("Rows: " + str(rows)) 
print("Columns: " + str(cols)) 

Rows: 370
Columns: 2


In [66]:
# convert dataframe to list
temp = df.values.tolist()

# for i in temp:
#     print(i[1])

In [90]:
# text=[]
# # remove inner brackets and quotes
# for t in temp:
#     nobrackets = str(t)[1:-1]
#     stripped = nobrackets.strip('"')
#     text.append(str(stripped)[1:-1])

In [67]:
text=[]

for i in temp:
    text.append(i[0])


In [69]:
section=[]
for i in range(len(text)):
    section.append(0)
len(section)
x=0


for i in range(x,len(text)):
    if 'ace inhibitors lower' not in text[i].lower():
        section[i]=1
        x+=1
    elif 'ace inhibitors lower' in text[i].lower():
        section[i]=2
        x+=1
#         print(text[i])
        break
    
for i in range(x,len(text)):
    if 'ethnic disparities' not in text[i].lower():
        section[i]=2
        x+=1
    elif 'ethnic disparities' in text[i].lower():
        section[i]=3
        x+=1
#         print(text[i])
        break

for i in range(x,len(text)):
    if 'need to see endocrinology' not in text[i].lower():
        section[i]=3
        x+=1
    elif 'need to see endocrinology' in text[i].lower():
        section[i]=4
        x+=1
#         print(text[i])
        break
    
for i in range(x,len(text)):
    if "association between ace's" not in text[i].lower():
        section[i]=4
        x+=1
    elif "association between ace's" in text[i].lower():
        section[i]=5
        x+=1
#         print(text[i])
        break

for i in range(x,len(text)):
    if "thank you so much, michel" not in text[i].lower():
        section[i]=5
        x+=1
    elif 'thank you so much, michel' in text[i].lower():
        break


In [70]:
section

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [85]:
lists = list(zip(section, text)) 


In [96]:
df

,section,text,time
0,1,All right.,1.160167
1,1,"Thanks, Doctor Thornton.",1.166833
2,1,Good morning and good morning before a holiday weekend.,1.183000
3,1,"So it's good to see some, uh, fases, even though maybe no one's going anywhere for the holiday.",1.232833
4,1,"I know I'm not, but that's okay.",1.312833
5,1,"So thanks for joining us this morning, and I I asked a colleague of mine to give a presentation today on, Ah, diabetes.",1.345167
6,1,"And the reason, um, you know, the Wuhan data suggested that the patients that had the worst outcome were those with cardiovascular disease and, um, you know, age over, you know, 50 60 70 and I wen...",1.487333
7,1,"Um, but what we noticed in our cohort of patients, the majority of patients that did poorly were actually diabetics and in potential obese diabetic.",1.758000
8,1,"So my question was, you know, what is there?",1.923333
9,1,Is there something specific about diabetes that's putting these patients at risk and do?,1.985833


In [92]:
q1=[]
q2=[]
q3=[]
q4=[]
q5=[]

for i in lists:
    if i[0]==1:
        q1.append(i[1])
    elif i[0]==2:
        q2.append(i[1])
    elif i[0]==3:
        q3.append(i[1])
    elif i[0]==4:
        q4.append(i[1])
    elif i[0]==5:
        q5.append(i[1])

In [68]:
questions_clean

[' do you think her risk was more increased due to obesity rather than diabetes?\n',
 ' any decreased lung expression of ACE2???\n',
 ' Do ACE inhibitors lower ACE1 or ACE2, or both?  Thank you.\n',
 ' Taking health and economy disparities out of the discussion, do you have any data suggesting that Hispanics may be at a higher risk of hospitalization or worse outcomes due to their known increased risk for metabolic syndrome and risk for diabetes?\n',
 ' Do these patients need to see endocrinology and cardiology outpatient one they are discharged?\n',
 ' Will the association between ACE2i and the outcome of COVID-19 supports giving non-diabetes patients with COVID-19 ACE2i?  \n']

In [71]:
questions_clean.remove(' any decreased lung expression of ACE2???\n')

In [73]:
questions_clean

[' do you think her risk was more increased due to obesity rather than diabetes?\n',
 ' Do ACE inhibitors lower ACE1 or ACE2, or both?  Thank you.\n',
 ' Taking health and economy disparities out of the discussion, do you have any data suggesting that Hispanics may be at a higher risk of hospitalization or worse outcomes due to their known increased risk for metabolic syndrome and risk for diabetes?\n',
 ' Do these patients need to see endocrinology and cardiology outpatient one they are discharged?\n',
 ' Will the association between ACE2i and the outcome of COVID-19 supports giving non-diabetes patients with COVID-19 ACE2i?  \n']

In [102]:
# f=open(clean/dataq_questions,'w') 
f=open('data/clean/2020-07-02 08.01.05 Critical Care for the COVID-19 Patient session 123702861_chat_questions.txt','w')


for q in range(len(questions_clean)):
    print(q)
    f.write('~~~~~~~~~~~~~'+'\n')
    f.write('Question'+str(q+1)+'\n')
    f.write('~~~~~~~~~~~~~'+'\n')
    f.write('\n')
    f.write('Q:' + questions_clean[q]+'\n')
    f.write('A: '+'\n')
    for i in lists:
        if i[0]==q+1:
            f.write(i[1]+'\n')
    f.write('\n')

f.close()

0
1
2
3
4


In [60]:
questions_clean

[' do you think her risk was more increased due to obesity rather than diabetes?\n',
 ' any decreased lung expression of ACE2???\n',
 ' Do ACE inhibitors lower ACE1 or ACE2, or both?  Thank you.\n',
 ' Taking health and economy disparities out of the discussion, do you have any data suggesting that Hispanics may be at a higher risk of hospitalization or worse outcomes due to their known increased risk for metabolic syndrome and risk for diabetes?\n',
 ' Do these patients need to see endocrinology and cardiology outpatient one they are discharged?\n',
 ' Will the association between ACE2i and the outcome of COVID-19 supports giving non-diabetes patients with COVID-19 ACE2i?  \n']

# Literature

In [151]:
import re

blogs=[]
for t in text_list:
    res=re.findall(r'(https?://\S+)', t)
    if bool(res)==True:
        if 'unm.edu' not in res[0]:
            if 'covid19_office_hours' not in res[0]:
                if 'youtube.com' not in res[0]:
                    if 'doi' not in res[0]:
                        print(res[0])
                        blogs.append(res[0])
                
        
len(blogs)

https://www.cdc.gov/coronavirus/2019-ncov/prevent-getting-sick/how-covid-spreads.html
https://www.scientificamerican.com/article/covid-19-is-now-the-third-leading-cause-of-death-in-the-u-s1/
https://uhfnyc.org/publications/publication/covid-19-ripple-effect-impact-covid-19-children-new-york-state/
https://webnew.ped.state.nm.us/wp-content/uploads/2020/09/NMPED_COVID19ResponseToolkit_ELEMpublic.pdf
https://services.aap.org/en/pages/2019-novel-coronavirus-covid-19-infections/clinical-guidance/covid-19-testing-guidance/
https://medium.com/@robinschoenthaler/covid-myth-busting-b5b26b22970f
https://www.theatlantic.com/ideas/archive/2020/10/schools-arent-superspreaders/616669/
https://www.politico.com/news/magazine/2020/09/25/how-covid-19-pandemic-ends-421122
https://www.epa.gov/coronavirus/air-cleaners-hvac-filters-and-coronavirus-covid-19
http://blog.jetblue.com/category/archives/coronavirus/
https://www.fda.gov/medical-devices/coronavirus-covid-19-and-medical-devices/uv-lights-and-lamps-u

12

In [152]:
articles=[]
for t in text_list:
    res=re.findall(r'(https?://\S+)', t)
    if bool(res)==True:
        if 'unm.edu' not in res[0]:
            if 'covid19_office_hours' not in res[0]:
                if 'youtube.com' not in res[0]:
                    if 'doi' in res[0]:
                        print(res[0])
                        articles.append(res[0])
                
        
len(articles)

https://doi.org/10.1056/NEJMoa2028436
https://doi.org/10.1001/jamainternmed.2020.6319
https://doi.org/10.1016/j.cmi.2020.09.052
https://doi.org/10.1056/NEJMoa2007764
http://dx.doi.org/10.15585/mmwr.mm6940e3
https://doi.org/10.1001/jama.2020.15139
https://doi.org/10.1186/s12985-020-01418-7


7

## Summary of blog posts

In [153]:
import pandas as pd
import newspaper
from newspaper import Article
from newspaper import fulltext
import requests
from gensim.summarization import summarize
from gensim.summarization import keywords
from docx import Document
from docx.text.paragraph import Paragraph

In [154]:
import docx

doc = docx.Document()
doc.save('data/clean/2020-10-12 12.03.10 Office Hours NM COVID-19 659261463_blogs.docx')

for link in blogs:
    try:
        # Pass link into article 
        article = Article(link)

        # Call the download and parse methods to download information 
        article.download()
        article.parse()
        html = requests.get(link).text

        # Grab article's body
        body = fulltext(html)

        # Grab article's info
        title=article.title
        date = article.publish_date

        # Run nlp method to extract the keywords and summary of the article
        article.nlp()

        # Split into sentences and remove empty spaces
        text=(body)
        sentences=text.split('\n')
        while('' in sentences) : 
            sentences.remove('') 

        # Create a single string
        article_string = ' '.join(sentences)

        # Summarize with Gensim
        if len(article_string)<4000:
            summary = summarize(article_string, word_count=200) # use wordcount if article is too short
        else:
            summary = summarize(article_string, ratio=0.10) # otherwise the output will include 5% of the total number of words
        summary_length = summary.split('.')

        # Get keywords
        key= keywords(article_string, lemmatize=True, words=10).split('\n')

        # Print to Word
        doc = Document('data/clean/2020-10-12 12.03.10 Office Hours NM COVID-19 659261463_blogs.docx')
        doc.add_heading(f'{title}', 1)
    #     doc.add_heading(f'Date: {date.month}/{date.day}/{date.year}', 6)
        paraKey = doc.add_heading('Keywords: ', 7)
        i=0
        while i<6:
            paraKey.add_run(key[i] + ', ') 
            i+=1
        paraKey.add_run(key[6] + '.')
        doc.add_heading(f'Source: {link}', 6)
    #     doc.add_heading(f'Length of original article: {len(sentences)} ; length of summary: {len(summary_length)}', 6)
        doc.add_heading(f'Summary:',6)
        doc.add_paragraph(summary)
        doc.add_page_break() 
        doc.save('data/clean/2020-10-12 12.03.10 Office Hours NM COVID-19 659261463_blogs.docx')
    except Exception as e:
        print(e)
        


Article `download()` failed with HTTPSConnectionPool(host='webnew.ped.state.nm.us', port=443): Max retries exceeded with url: /wp-content/uploads/2020/09/NMPED_COVID19ResponseToolkit_ELEMpublic.pdf (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x7fbcc7e1c150>, 'Connection to webnew.ped.state.nm.us timed out. (connect timeout=7)')) on URL https://webnew.ped.state.nm.us/wp-content/uploads/2020/09/NMPED_COVID19ResponseToolkit_ELEMpublic.pdf


In [155]:
# from docx2pdf import convert
# convert("data/clean/2020-10-12 12.03.10 Office Hours NM COVID-19 659261463_blogs.docx", 
#         "data/clean/2020-10-12 12.03.10 Office Hours NM COVID-19 659261463_blogs.pdf")

# Speaker 2

## Import text file

In [5]:
# Read text file into a dataframe
df = pd.read_csv(textfile/video2, sep="\t",
                 header=0,
                 names=["text", "minute"],
                 index_col=False)

In [6]:
df

,text,minute
0,And And what is it about diabetes that is making this?,8.626333
1,"Um, such a risk factor for patients with COVID.",8.680833
2,So with that point Doctor Bouchonville?,8.734333
3,I would welcome you.,8.773167
4,Thank you for presenting this morning.,8.794500
5,"And, um, we're excited to learn from you.",8.823167
6,My pleasure.,8.868333
7,"Thank you so much, Doctor Harkins what a case.",8.879167
8,"Um, I'm gonna go ahead and share my screen here so I can pull up my slides.",8.932333
9,"And, uh, let's see if your screen is like mine.",9.109000


In [7]:
# Drop empty rows
df.dropna(
    axis=0,
    how='any',
    thresh=None,
    subset=None,
    inplace=True
)

In [8]:
df.head()

,text,minute
0,And And what is it about diabetes that is making this?,8.626333
1,"Um, such a risk factor for patients with COVID.",8.680833
2,So with that point Doctor Bouchonville?,8.734333
3,I would welcome you.,8.773167
4,Thank you for presenting this morning.,8.794500


In [9]:
# displaying the number of rows and columns 
rows = df.shape[0] 
cols = df.shape[1] 
print("Rows: " + str(rows)) 
print("Columns: " + str(cols)) 

Rows: 145
Columns: 2


In [10]:
# convert dataframe to list
temp = df.values.tolist()

# for i in temp:
#     print(i[1])

In [90]:
# text=[]
# # remove inner brackets and quotes
# for t in temp:
#     nobrackets = str(t)[1:-1]
#     stripped = nobrackets.strip('"')
#     text.append(str(stripped)[1:-1])

In [11]:
text=[]
time=[]
for i in temp:
    text.append(i[0])
    time.append(i[1])    

In [12]:
# sanity check
for i in range(10):
    print(text[i] + ' Starting time: ' + str(time[i]))

And And what is it about diabetes that is making this? Starting time: 8.626333333333333
Um, such a risk factor for patients with COVID. Starting time: 8.680833333333334
So with that point Doctor Bouchonville? Starting time: 8.734333333333332
I would welcome you. Starting time: 8.773166666666667
Thank you for presenting this morning. Starting time: 8.7945
And, um, we're excited to learn from you. Starting time: 8.823166666666667
My pleasure. Starting time: 8.868333333333334
Thank you so much, Doctor Harkins what a case. Starting time: 8.879166666666666
Um, I'm gonna go ahead and share my screen here so I can pull up my slides. Starting time: 8.932333333333334
And, uh, let's see if your screen is like mine. Starting time: 9.109


In [28]:


'Conclusions'

section=[]
for i in range(len(text)):
    section.append(0)
len(section)
x=0
for i in range(len(text)):
    if 'how can we quantify the increased risk' not in text[i].lower():
        section[i]=1
#         print(text[i])
        x+=1
    elif 'how can we quantify the increased risk' in text[i].lower():
        section[i]=2
        x+=1
        print(x)
        break

for i in range(x,len(text)):
    if 'how does glycemia control impact' not in text[i].lower():
        section[i]=2
        x+=1
    elif 'how does glycemia control impact' in text[i].lower():
        section[i]=3
        x+=1
        break
        
for i in range(x,len(text)):
    if 'why do people outside of hyperglycemia' not in text[i].lower():
        section[i]=3
        x+=1
    elif 'why do people outside of hyperglycemia' in text[i].lower():
        section[i]=4
        x+=1
        break        

for i in range(x,len(text)):
    if 'summarize everything' not in text[i].lower():
        section[i]=4
        x+=1
    elif 'summarize everything' in text[i].lower():
        section[i]=5
        x+=1
        break        
        
for i in range(x,len(text)):
#     print(text[i])
    section[i]=5

49


In [29]:
section

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 5,
 5,
 5,
 5,
 5,
 5,
 5]

In [30]:
lists = list(zip(section, text, time)) 
df = pd.DataFrame(lists, columns = ['section', 'text', 'time']) 

In [31]:
df

,section,text,time
0,1,And And what is it about diabetes that is making this?,8.626333
1,1,"Um, such a risk factor for patients with COVID.",8.680833
2,1,So with that point Doctor Bouchonville?,8.734333
3,1,I would welcome you.,8.773167
4,1,Thank you for presenting this morning.,8.794500
5,1,"And, um, we're excited to learn from you.",8.823167
6,1,My pleasure.,8.868333
7,1,"Thank you so much, Doctor Harkins what a case.",8.879167
8,1,"Um, I'm gonna go ahead and share my screen here so I can pull up my slides.",8.932333
9,1,"And, uh, let's see if your screen is like mine.",9.109000


In [32]:
start_time = df.groupby(['section'], sort=False)['time'].max()
start_time.head()
# start_time.to_csv(clean/'speaker1_sections.txt', sep='\t', index=False, header=False)

section
1    13.635667
2    16.514000
3    18.275667
4    26.557500
5    27.570667
Name: time, dtype: float64

In [34]:
copy = df[['text']].copy()
copy
s1 = df['section']==1
df_1 = df.loc[s1]
df_1 = df_1[['text']]
df_1
df_1.to_csv(clean/'2020-07-02 08.01.05 Critical Care for the COVID-19 Patient session 123702861_speaker2_1.txt', sep='\t', index=False, header=False)

s2 = df['section']==2
df_2 = df.loc[s2]
df_2 = df_2[['text']]
df_2.to_csv(clean/'2020-07-02 08.01.05 Critical Care for the COVID-19 Patient session 123702861_speaker2_2.txt', sep='\t', index=False, header=False)

s3 = df['section']==3
df_3 = df.loc[s3]
df_3 = df_3[['text']]
df_3.to_csv(clean/'2020-07-02 08.01.05 Critical Care for the COVID-19 Patient session 123702861_speaker2_3.txt', sep='\t', index=False, header=False)

s4 = df['section']==4
df_4 = df.loc[s4]
df_4 = df_4[['text']]
df_4.to_csv(clean/'2020-07-02 08.01.05 Critical Care for the COVID-19 Patient session 123702861_speaker2_4.txt', sep='\t', index=False, header=False)

s5 = df['section']==5
df_5 = df.loc[s5]
df_5 = df_5[['text']]
df_5.to_csv(clean/'2020-07-02 08.01.05 Critical Care for the COVID-19 Patient session 123702861_speaker2_5.txt', sep='\t', index=False, header=False)